In [371]:
import pandas as pd
import numpy as np
import re
import string

# Major Crime Data
Source: https://data.torontopolice.on.ca/datasets/TorontoPS::major-crime-indicators-open-data/about

In [471]:
crime = pd.read_csv('data/crime/Major_Crime_Indicators_Open_Data.csv', index_col='OBJECTID')
crime.head()

,X,Y,EVENT_UNIQUE_ID,REPORT_DATE,OCC_DATE,REPORT_YEAR,REPORT_MONTH,REPORT_DAY,REPORT_DOY,REPORT_DOW,...,UCR_CODE,UCR_EXT,OFFENCE,MCI_CATEGORY,HOOD_158,NEIGHBOURHOOD_158,HOOD_140,NEIGHBOURHOOD_140,LONG_WGS84,LAT_WGS84
OBJECTID,,,,,,,,,,,,,,,,,,,,,
1,-8.809036e+06,5.431523e+06,GO-20141260264,2014/01/01 05:00:00+00,2014/01/01 05:00:00+00,2014,January,1,1,Wednesday,...,1430,100,Assault,Assault,143,West Rouge,131,Rouge (131),-79.132915,43.780413
2,-8.814320e+06,5.435514e+06,GO-20141260033,2014/01/01 05:00:00+00,2013/12/31 05:00:00+00,2014,January,1,1,Wednesday,...,1430,100,Assault,Assault,144,Morningside Heights,131,Rouge (131),-79.180387,43.806289
3,-8.832825e+06,5.419631e+06,GO-20141259834,2014/01/01 05:00:00+00,2014/01/01 05:00:00+00,2014,January,1,1,Wednesday,...,1420,100,Assault With Weapon,Assault,55,Thorncliffe Park,55,Thorncliffe Park (55),-79.346615,43.703234
4,-8.847292e+06,5.429042e+06,GO-20141264084,2014/01/01 05:00:00+00,2013/12/31 05:00:00+00,2014,January,1,1,Wednesday,...,1420,110,Assault Bodily Harm,Assault,27,York University Heights,27,York University Heights (27),-79.476579,43.764317
5,6.327780e-09,5.664924e-09,GO-20141260577,2014/01/01 05:00:00+00,2014/01/01 05:00:00+00,2014,January,1,1,Wednesday,...,1610,180,Robbery - Swarming,Robbery,NSA,NSA,NSA,NSA,0.000000,0.000000


In [472]:
crime.columns

Index(['X', 'Y', 'EVENT_UNIQUE_ID', 'REPORT_DATE', 'OCC_DATE', 'REPORT_YEAR',
       'REPORT_MONTH', 'REPORT_DAY', 'REPORT_DOY', 'REPORT_DOW', 'REPORT_HOUR',
       'OCC_YEAR', 'OCC_MONTH', 'OCC_DAY', 'OCC_DOY', 'OCC_DOW', 'OCC_HOUR',
       'DIVISION', 'LOCATION_TYPE', 'PREMISES_TYPE', 'UCR_CODE', 'UCR_EXT',
       'OFFENCE', 'MCI_CATEGORY', 'HOOD_158', 'NEIGHBOURHOOD_158', 'HOOD_140',
       'NEIGHBOURHOOD_140', 'LONG_WGS84', 'LAT_WGS84'],
      dtype='object')

In [473]:
# The OFFENCE column has very specific descriptions of the crimes, we are only interested in the larger category
# As for the report date columns, we are interested in when it happened - not when it was reported.
crime.drop(columns=['REPORT_DATE', 'REPORT_YEAR', 'REPORT_MONTH', 'REPORT_DAY', 'REPORT_DOY', 'REPORT_DOW', 'REPORT_HOUR', 'OFFENCE', 'HOOD_158', 'HOOD_140'], inplace=True)

In [474]:
# Create a new DataFrame with the unique neighbourhood158 values and their corresponding updated names
neighbourhood_mapping = crime.groupby('NEIGHBOURHOOD_158')['NEIGHBOURHOOD_140'].unique().reset_index()

new_neighbourhood_df = pd.DataFrame({
    'past_neighbourhood_158': neighbourhood_mapping['NEIGHBOURHOOD_158'],
    'neighbourhood': neighbourhood_mapping['NEIGHBOURHOOD_140'].apply(lambda x: x[0]) # assuming there's only one updated name
})

new_neighbourhood_df

,past_neighbourhood_158,neighbourhood
0,Agincourt North,Agincourt North (129)
1,Agincourt South-Malvern West,Agincourt South-Malvern West (128)
2,Alderwood,Alderwood (20)
3,Annex,Annex (95)
4,Avondale,Willowdale East (51)
...,...,...
154,Yonge-Doris,Willowdale East (51)
155,Yonge-Eglinton,Yonge-Eglinton (100)
156,Yonge-St.Clair,Yonge-St.Clair (97)
157,York University Heights,York University Heights (27)


In [475]:
# Remove the number and brackets at the end of the neighbourhood name
def clean_string(text):
    pattern = r'\s*\(\d+\)$'
    cleaned = re.sub(pattern, '', text)

    return cleaned

In [476]:
new_neighbourhood_df['neighbourhood'] = new_neighbourhood_df['neighbourhood'].apply(clean_string)

In [477]:
crime['NEIGHBOURHOOD_140'] = crime['NEIGHBOURHOOD_140'].apply(clean_string)

In [478]:
# Now we can drop the old neighbourhood_158 column
crime.drop(columns=['NEIGHBOURHOOD_158'], inplace=True)

In [479]:
crime.columns

Index(['X', 'Y', 'EVENT_UNIQUE_ID', 'OCC_DATE', 'OCC_YEAR', 'OCC_MONTH',
       'OCC_DAY', 'OCC_DOY', 'OCC_DOW', 'OCC_HOUR', 'DIVISION',
       'LOCATION_TYPE', 'PREMISES_TYPE', 'UCR_CODE', 'UCR_EXT', 'MCI_CATEGORY',
       'NEIGHBOURHOOD_140', 'LONG_WGS84', 'LAT_WGS84'],
      dtype='object')

In [480]:
# Converting the occurence data to a datetime variable for easy filtering
crime['OCC_DATE'] = pd.to_datetime(crime['OCC_DATE'], format='%Y/%m/%d %H:%M:%S+00')

In [481]:
# Filtering out crimes that happened before 01/01/2013 and after 31/12/2023
start_date = pd.to_datetime('2013-01-01')
end_date = pd.to_datetime('2023-12-31')

crime = crime[(crime['OCC_DATE'] >= start_date) & (crime['OCC_DATE'] <= end_date)]

In [482]:
crime = crime.sort_values(by='OCC_DATE')

In [483]:
crime['NEIGHBOURHOOD_140'] = crime['NEIGHBOURHOOD_140'].str.lower()
crime['NEIGHBOURHOOD_140'] = crime['NEIGHBOURHOOD_140'].map(lambda x: x.translate(str.maketrans('', '', string.punctuation)))

In [484]:
new_neighbourhood_df['neighbourhood'] = new_neighbourhood_df['neighbourhood'].str.lower()
new_neighbourhood_df['neighbourhood'] = new_neighbourhood_df['neighbourhood'].map(lambda x: x.translate(str.maketrans('', '', string.punctuation)))


In [485]:
# Create separate data frames for each year
crime_by_date = crime.groupby(crime['OCC_DATE'].dt.year)
yearly_crime_dfs = {year: group for year, group in crime_by_date}
# access each year by -> yearly_crime_dfs[x] where x is 2013, 2014, ..., 2023

In [486]:
years = [2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]

In [487]:
monthly_summaries = {}
crime_types = ['Assault', 'Break and Enter', 'Theft Over', 'Robbery', 'Auto Theft']

# Iterate through each yearly DataFrame
for year, df_yearly in yearly_crime_dfs.items():
    df_yearly['MONTH'] = df_yearly['OCC_DATE'].dt.month
    
    # Group the yearly DataFrame by month and by crime category, then count the number of crimes
    monthly_summary_yearly = df_yearly.groupby(['MONTH', 'MCI_CATEGORY']).size().unstack(fill_value=0).reset_index()
    
    # Add a 'YEAR' column to the monthly summary DataFrame
    monthly_summary_yearly['YEAR'] = year
    monthly_summary_yearly = monthly_summary_yearly[['YEAR', 'MONTH'] + crime_types]
    
    monthly_summaries[year] = monthly_summary_yearly.to_dict(orient='records')

In [488]:
# crimes by neighbourhood
yearly_neighborhood_summaries = {}

for year, df_yearly in yearly_crime_dfs.items():
    neighborhood_summary = df_yearly['NEIGHBOURHOOD_140'].value_counts().reset_index()
    neighborhood_summary.columns = ['NEIGHBOURHOOD_140', 'num_crimes']
    
    yearly_neighborhood_summaries[year] = neighborhood_summary

In [489]:
# save as .csv
for year in years:
    yearly_crime_dfs[year].to_csv('data/crime/by_year/all_crimes' + str(year) + '.csv')
    pd.DataFrame(monthly_summaries[year]).to_csv('data/crime/by_year_month/crime_summary' + str(year) + '.csv')
    yearly_neighborhood_summaries[year].to_csv('data/crime/by_neighbourhood/' + str(year) + 'crimes_per_neighbourhood.csv')

In [490]:
# this file was too big to keep in the repo
crime.to_csv('data/crime/major_crimes.csv')
new_neighbourhood_df.to_csv('data/neighbourhoods.csv')

## Clean neighbourhood column

# Neighbourhood Polygon Data
Source: https://open.toronto.ca/dataset/neighbourhood-profiles/

In [ ]:
neighbourhoods_140 = pd.read_csv('data/neighbourhoods_140.csv', index_col=0)
neighbourhoods_140.head(2)

In [ ]:
neighbourhoods_140.columns

In [ ]:
neighbourhoods_140.drop(columns=['PARENT_AREA_ID', 'AREA_SHORT_CODE', 'AREA_LONG_CODE', 'AREA_DESC', 'CLASSIFICATION_CODE'], inplace=True)

In [ ]:
neighbourhoods_140['AREA_NAME'] = neighbourhoods_140['AREA_NAME'].apply(clean_string)

In [ ]:
neighbourhoods_140.to_csv('data/neighbourhoods_140.csv')

# Census 2016

In [372]:
census2016 = pd.read_csv('data/raw/neighbourhood-profiles-2016.csv', index_col=0)

In [373]:
census2016.head()

,Category,Topic,Data Source,Characteristic,City of Toronto,Agincourt North,Agincourt South-Malvern West,Alderwood,Annex,Banbury-Don Mills,...,Willowdale West,Willowridge-Martingrove-Richview,Woburn,Woodbine Corridor,Woodbine-Lumsden,Wychwood,Yonge-Eglinton,Yonge-St.Clair,York University Heights,Yorkdale-Glen Park
_id,,,,,,,,,,,,,,,,,,,,,
1,Neighbourhood Information,Neighbourhood Information,City of Toronto,Neighbourhood Number,NaN,129,128,20,95,42,...,37,7,137,64,60,94,100,97,27,31
2,Neighbourhood Information,Neighbourhood Information,City of Toronto,TSNS2020 Designation,NaN,No Designation,No Designation,No Designation,No Designation,No Designation,...,No Designation,No Designation,NIA,No Designation,No Designation,No Designation,No Designation,No Designation,NIA,Emerging Neighbourhood
3,Population,Population and dwellings,Census Profile 98-316-X2016001,"Population, 2016","2,731,571","29,113","23,757","12,054","30,526","27,695",...,"16,936","22,156","53,485","12,541","7,865","14,349","11,817","12,528","27,593","14,804"
4,Population,Population and dwellings,Census Profile 98-316-X2016001,"Population, 2011","2,615,060","30,279","21,988","11,904","29,177","26,918",...,"15,004","21,343","53,350","11,703","7,826","13,986","10,578","11,652","27,713","14,687"
5,Population,Population and dwellings,Census Profile 98-316-X2016001,Population Change 2011-2016,4.50%,-3.90%,8.00%,1.30%,4.60%,2.90%,...,12.90%,3.80%,0.30%,7.20%,0.50%,2.60%,11.70%,7.50%,-0.40%,0.80%


In [374]:
# drop the following columns as they do not exist in the census 2021 file
census2016.drop(columns=['Category', 'Topic', 'Data Source'], inplace=True)

In [375]:
# Now transpose the dataframe so the columns are rows and vice versa
census2016.set_index('Characteristic', inplace=True)
census2016 = census2016.transpose()
census2016.head()

Characteristic,Neighbourhood Number,TSNS2020 Designation,"Population, 2016","Population, 2011",Population Change 2011-2016,Total private dwellings,Private dwellings occupied by usual residents,Population density per square kilometre,Land area in square kilometres,Children (0-14 years),...,Intraprovincial migrants,Interprovincial migrants,External migrants,Total - Mobility status 5 years ago - 25% sample data,Non-movers,Movers,Non-migrants,Migrants,Interprovincial migrants,External migrants
City of Toronto,NaN,NaN,"2,731,571","2,615,060",4.50%,"1,179,057","1,112,929","4,334",630.2,"398,135",...,"49,990","15,780","59,945","2,556,120","1,516,110","1,040,015","639,060","400,950","42,985","216,835"
Agincourt North,129,No Designation,"29,113","30,279",-3.90%,"9,371","9,120","3,929",7.41,"3,840",...,275,75,605,"27,490","18,865","8,610","5,445","3,170",135,"2,280"
Agincourt South-Malvern West,128,No Designation,"23,757","21,988",8.00%,"8,535","8,136","3,034",7.83,"3,075",...,320,90,490,"22,325","13,565","8,775","5,610","3,145",220,"2,170"
Alderwood,20,No Designation,"12,054","11,904",1.30%,"4,732","4,616","2,435",4.95,"1,760",...,220,40,70,"11,370","8,235","3,130","2,200",925,70,245
Annex,95,No Designation,"30,526","29,177",4.60%,"18,109","15,934","10,863",2.81,"2,360",...,900,385,835,"27,715","12,980","14,735","8,340","6,390","1,310","2,460"


In [376]:
income = census2016.iloc[:,1037:1054]

In [377]:
income['$15,000 to $19,999'] = census2016.iloc[:, 1021]

/var/folders/20/k5yhw6n974v_ccqvnn3jmm2w0000gq/T/ipykernel_45240/2495638670.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  income['$15,000 to $19,999'] = census2016.iloc[:, 1021]


In [378]:
income.columns

Index(['Total - Household total income groups in 2015 for private households - 100% data',
       '  Under $5,000', '  $5,000 to $9,999', '  $10,000 to $14,999',
       '  $20,000 to $24,999', '  $25,000 to $29,999', '  $30,000 to $34,999',
       '  $35,000 to $39,999', '  $40,000 to $44,999', '  $45,000 to $49,999',
       '  $50,000 to $59,999', '  $60,000 to $69,999', '  $70,000 to $79,999',
       '  $80,000 to $89,999', '  $90,000 to $99,999', '  $100,000 and over',
       '    $200,000 and over', '$15,000 to $19,999'],
      dtype='object', name='Characteristic')

In [379]:
income.columns = income.columns.str.strip()

In [380]:
income.columns

Index(['Total - Household total income groups in 2015 for private households - 100% data',
       'Under $5,000', '$5,000 to $9,999', '$10,000 to $14,999',
       '$20,000 to $24,999', '$25,000 to $29,999', '$30,000 to $34,999',
       '$35,000 to $39,999', '$40,000 to $44,999', '$45,000 to $49,999',
       '$50,000 to $59,999', '$60,000 to $69,999', '$70,000 to $79,999',
       '$80,000 to $89,999', '$90,000 to $99,999', '$100,000 and over',
       '$200,000 and over', '$15,000 to $19,999'],
      dtype='object', name='Characteristic')

In [381]:
# need to aggregate certain income columns to match other datasets
income.loc[:,'$20,000 to $29,000'] = income['$20,000 to $24,999'] + income['$25,000 to $29,999']
income.drop(columns=['$20,000 to $24,999', '$25,000 to $29,999'], inplace=True)

income.loc[:,'$30,000 to $39,000'] = income['$30,000 to $34,999'] + income['$35,000 to $39,999']
income.drop(columns=['$30,000 to $34,999', '$35,000 to $39,999'], inplace=True)

income.loc[:,'$40,000 to $49,000'] = income['$40,000 to $44,999'] + income['$45,000 to $49,999']
income.drop(columns=['$40,000 to $44,999', '$45,000 to $49,999'], inplace=True)

income.loc[:,'$60,000 to $79,000'] = income['$60,000 to $69,999'] + income['$70,000 to $79,999']
income.drop(columns=['$60,000 to $69,999', '$70,000 to $79,999'], inplace=True)

income.loc[:,'$80,000 to $99,000'] = income['$80,000 to $89,999'] + income['$90,000 to $99,999']
income.drop(columns=['$80,000 to $89,999', '$90,000 to $99,999'], inplace=True)

income.loc[:,'$100,000 and over'] = income['$100,000 and over'] + income['$200,000 and over']
income.drop(columns=['$200,000 and over'], inplace=True)

/var/folders/20/k5yhw6n974v_ccqvnn3jmm2w0000gq/T/ipykernel_45240/2251589098.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  income.loc[:,'$20,000 to $29,000'] = income['$20,000 to $24,999'] + income['$25,000 to $29,999']
/var/folders/20/k5yhw6n974v_ccqvnn3jmm2w0000gq/T/ipykernel_45240/2251589098.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  income.drop(columns=['$20,000 to $24,999', '$25,000 to $29,999'], inplace=True)
/var/folders/20/k5yhw6n974v_ccqvnn3jmm2w0000gq/T/ipykernel_45240/2251589098.py:5: SettingWithCopyWarning: 
A value is trying to be set o

In [382]:
education = census2016.iloc[:, 1702:1707]
education.columns = education.columns.str.strip()

In [383]:
education['Postsecondary certificate, diploma or degree'] = census2016.iloc[:, 1675]

/var/folders/20/k5yhw6n974v_ccqvnn3jmm2w0000gq/T/ipykernel_45240/1408118973.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  education['Postsecondary certificate, diploma or degree'] = census2016.iloc[:, 1675]


In [384]:
education.columns

Index(['Total - Highest certificate, diploma or degree for the population aged 15 years and over in private households - 25% sample data',
       'No certificate, diploma or degree',
       'Secondary (high) school diploma or equivalency certificate',
       'Trades certificate or diploma other than Certificate of Apprenticeship or Certificate of Qualification',
       'Certificate of Apprenticeship or Certificate of Qualification',
       'Postsecondary certificate, diploma or degree'],
      dtype='object', name='Characteristic')

In [385]:
education['Trades certificate or diploma other than Certificate of Apprenticeship or Certificate of Qualification'] = education['Trades certificate or diploma other than Certificate of Apprenticeship or Certificate of Qualification'] + education['Certificate of Apprenticeship or Certificate of Qualification']
education.drop(columns=['Certificate of Apprenticeship or Certificate of Qualification', 'Trades certificate or diploma other than Certificate of Apprenticeship or Certificate of Qualification'], inplace=True)

/var/folders/20/k5yhw6n974v_ccqvnn3jmm2w0000gq/T/ipykernel_45240/1638384041.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  education['Trades certificate or diploma other than Certificate of Apprenticeship or Certificate of Qualification'] = education['Trades certificate or diploma other than Certificate of Apprenticeship or Certificate of Qualification'] + education['Certificate of Apprenticeship or Certificate of Qualification']
/var/folders/20/k5yhw6n974v_ccqvnn3jmm2w0000gq/T/ipykernel_45240/1638384041.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ed

In [386]:
# There are lots of trailing white spaces which will affect the selection of columns so that is being taken out
census2016.columns = [col.strip() for col in census2016.columns]

In [387]:
pop_by_age_group = [
    'Male: 0 to 04 years',
    'Male: 05 to 09 years',
    'Male: 10 to 14 years',
    'Male: 15 to 19 years',
    'Male: 20 to 24 years',
    'Male: 25 to 29 years',
    'Male: 30 to 34 years',
    'Male: 35 to 39 years',
    'Male: 40 to 44 years',
    'Male: 45 to 49 years',
    'Male: 50 to 54 years',
    'Male: 55 to 59 years',
    'Male: 60 to 64 years',
    'Male: 65 to 69 years',
    'Male: 70 to 74 years',
    'Male: 75 to 79 years',
    'Female: 10 to 14 years',
    'Male: 80 to 84 years',
    'Male: 85 to 89 years',
    'Male: 90 to 94 years',
    'Male: 95 to 99 years',
    'Male: 100 years and over',
    'Female: 0 to 04 years',
    'Female: 05 to 09 years',
    'Female: 15 to 19 years',
    'Female: 20 to 24 years',
    'Female: 25 to 29 years',
    'Female: 30 to 34 years',
    'Female: 35 to 39 years',
    'Female: 40 to 44 years',
    'Female: 45 to 49 years',
    'Female: 50 to 54 years',
    'Female: 55 to 59 years',
    'Female: 60 to 64 years',
    'Female: 65 to 69 years',
    'Female: 70 to 74 years',
    'Female: 75 to 79 years',
    'Female: 80 to 84 years',
    'Female: 85 to 89 years',
    'Female: 90 to 94 years',
    'Female: 95 to 99 years',
    'Female: 100 years and over'
]

characteristic_items = ['Single-detached house',
    'Apartment in a building that has five or more storeys',
    'Other attached dwelling',
    'Average household size',
    'Married or living common law',
    'Not married and not living common law',
    'Lone-parent census families in private households',
    'One-person households',
    'Two-or-more person non-census-family households',
    'Prevalence of low income based on the Low-income measure, after tax (LIM-AT) (%)',
    'Non-immigrants',
    'Immigrants',
    'Owner',
    'Renter',
    'One person or fewer per room',
    'More than 1 person per room',
    'Suitable',
    'Not suitable',
    'Total population aged 15 years and over by work activity during the reference year - 25% sample data',
    'Did not work',
    'Worked',
    'Car, truck, van - as a driver',
    'Car, truck, van - as a passenger',
    'Public transit',
    'Walked',
    'Bicycle',
    'Other method'
    ]

keep = ['Characteristic'] + characteristic_items

# this is to aggregate all ages
population = census2016.loc[:, census2016.columns.isin(pop_by_age_group)]
census2016 = census2016.loc[:, census2016.columns.isin(keep)]
census2016.head()

,Single-detached house,Apartment in a building that has five or more storeys,Other attached dwelling,Average household size,Married or living common law,Not married and not living common law,Lone-parent census families in private households,One-person households,Two-or-more person non-census-family households,"Prevalence of low income based on the Low-income measure, after tax (LIM-AT) (%)",...,Not suitable,Total population aged 15 years and over by work activity during the reference year - 25% sample data,Did not work,Worked,"Car, truck, van - as a driver","Car, truck, van - as a passenger",Public transit,Walked,Bicycle,Other method
City of Toronto,"269,675","493,275","349,880",2.42,"1,181,360","1,152,075","152,595","359,960","68,010",20.2,...,"134,825","2,294,790","791,795","1,502,995","575,255","57,170","463,000","107,665","34,355","13,610"
Agincourt North,3345,2120,3645,3.16,14200,11075,1650,1350,290,21.3,...,"1,315","25,000","11,060","13,940","7,155",930,"3,350",265,70,45
Agincourt South-Malvern West,2790,3145,2200,2.88,11205,9475,1360,1615,445,22.6,...,970,"20,400","8,270","12,140","6,135",665,"2,985",280,35,65
Alderwood,2840,255,1525,2.6,5735,4550,670,1105,185,8.2,...,325,"10,265","3,245","7,015","4,090",355,"1,285",195,65,65
Annex,645,8165,7125,1.8,12275,15890,895,7885,1430,18.3,...,800,"26,305","6,925","19,365","3,290",290,"6,200","3,200","1,675",225


In [388]:
# Rename the 0 to 04 year age groups for simplicity in code
population.rename(columns={'Male: 0 to 04 years': 'Male: 00 to 04 years', 'Female: 0 to 04 years': 'Female: 00 to 04 years'}, inplace=True)

In [389]:
for age_group in range(0, 101, 5):
    # Define column names for male and female
    male_col_name = f"Male: {age_group:02d} to {age_group + 4:02d} years"
    female_col_name = f"Female: {age_group:02d} to {age_group + 4:02d} years"
    
    if male_col_name in population.columns and female_col_name in population.columns:
        population[f"Population for {age_group:02d} to {age_group + 4:02d} years"] = population[male_col_name] + population[female_col_name]
        
        population.drop([male_col_name, female_col_name], axis=1, inplace=True)

In [390]:
population['Population for 100 years and over'] = population['Female: 100 years and over'] + population['Male: 100 years and over']
population.drop(columns=['Female: 100 years and over', 'Male: 100 years and over'], inplace=True)

In [391]:
# Merge together the two dataframes
census2016 = pd.concat([census2016, population], axis=1)
census2016 = pd.concat([census2016, income], axis=1)
census2016 = pd.concat([census2016, education], axis=1)

In [392]:
for col in census2016.columns:
    print(col)

Single-detached house
Apartment in a building that has five or more storeys
Other attached dwelling
Average household size
Married or living common law
Not married and not living common law
Lone-parent census families in private households
One-person households
Two-or-more person non-census-family households
Prevalence of low income based on the Low-income measure, after tax (LIM-AT) (%)
Non-immigrants
Immigrants
Owner
Renter
One person or fewer per room
More than 1 person per room
Suitable
Not suitable
Total population aged 15 years and over by work activity during the reference year - 25% sample data
Did not work
Worked
Car, truck, van - as a driver
Car, truck, van - as a passenger
Public transit
Walked
Bicycle
Other method
Population for 00 to 04 years
Population for 05 to 09 years
Population for 10 to 14 years
Population for 15 to 19 years
Population for 20 to 24 years
Population for 25 to 29 years
Population for 30 to 34 years
Population for 35 to 39 years
Population for 40 to 44 

In [393]:
census2016.index = census2016.index.str.lower()
census2016.index = census2016.index.map(lambda x: x.translate(str.maketrans('', '', string.punctuation)))

In [427]:
# shorten column names

census2016.rename(columns={'Single-detached house': 'single_detached_house', 
                           'Apartment in a building that has five or more storeys': 'apart_5_plus',
                           'Other attached dwelling': 'other_dwelling',
                           'Average household size' : 'avg_household_size',
                           'Married or living common law' : ' married',
                           'Not married and not living common law' : 'not_married',
                           'Lone-parent census families in private households' : 'single_parents',
                           'One-person households' : 'one_person_household',
                           'Two-or-more person non-census-family households' : 'two_plus_person_household',
                           'Prevalence of low income based on the Low-income measure, after tax (LIM-AT) (%)' : 'low_income_percent',
                           'Non-immigrants' : 'non_immigrants',
                           'One person or fewer per room' : 'own_room',
                           'More than 1 person per room' : 'sharing_room',
                           'Suitable' : 'suitable_housing',
                           'Not suitable' : 'not_suitable_housing',
                           'Total population aged 15 years and over by work activity during the reference year - 25% sample data' : 'total_work_activity',
                           'Did not work' : 'unemployed',
                           'Worked' : 'employed',
                            'Car, truck, van - as a driver' : 'commute_drives',
                            'Car, truck, van - as a passenger' : 'commute_passenger',
                            'Public transit' : 'commute_public_transport',
                            'Walked' : 'commute_walk',
                            'Bicycle' : 'commute_cycle',
                            'Other method' : 'commute_other',
                            'Total - Household total income groups in 2015 for private households - 100% data' : 'household_income',
                            'Under $5,000' : 'income_under_5000',
                            '$5,000 to $9,999' : 'income_between_5000_9999',
                            '$10,000 to $14,999' : 'income_between_10000_14999',
                            '$15,000 to $19,999' : 'income_between_15000_19999',
                            '$20,000 to $29,000' : 'income_between_20000_29999',
                            '$30,000 to $39,000' : 'income_between_30000_39999',
                            '$40,000 to $49,000' : 'income_between_40000_49999',
                            '$50,000 to $59,999' : 'income_between_50000_59999',
                            '$60,000 to $79,000' : 'income_between_60000_79999',
                            '$80,000 to $99,000' : 'income_between_80000_99999',
                            '$100,000 and over' : 'income_above_100k',
                            'Total - Highest certificate, diploma or degree for the population aged 15 years and over in private households - 25% sample data' : 'total_highest_education',
                            'No certificate, diploma or degree' : 'no_certificate_diploma_degree',
                            'Secondary (high) school diploma or equivalency certificate' : 'highschool_diploma',
                            'Postsecondary certificate, diploma or degree': 'post_secondary_diploma'
                           }, inplace=True)

In [395]:
for age_group in range(0, 99, 5):
    census2016.rename(columns={f"Population for {age_group:02d} to {age_group+4:02d} years" : f"pop_{age_group}_to_{age_group+4}"}, inplace=True)

In [428]:
census2016.rename(columns={"Population for 100 years and over" : "pop_100_plus"}, inplace=True)

In [430]:
census2016.columns = census2016.columns.str.lower()
census2016.columns = census2016.columns.str.strip()

In [436]:
census2016.columns

Index(['single_detached_house', 'apart_5_plus', 'other_dwelling',
       'avg_household_size', 'married', 'not_married', 'single_parents',
       'one_person_household', 'two_plus_person_household',
       'low_income_percent', 'non_immigrants', 'immigrants', 'owner', 'renter',
       'own_room', 'sharing_room', 'suitable_housing', 'not_suitable_housing',
       'total_work_activity', 'unemployed', 'employed', 'commute_drives',
       'commute_passenger', 'commute_public_transport', 'commute_walk',
       'commute_cycle', 'commute_other', 'pop_0_to_4', 'pop_5_to_9',
       'pop_10_to_14', 'pop_15_to_19', 'pop_20_to_24', 'pop_25_to_29',
       'pop_30_to_34', 'pop_35_to_39', 'pop_40_to_44', 'pop_45_to_49',
       'pop_50_to_54', 'pop_55_to_59', 'pop_60_to_64', 'pop_65_to_69',
       'pop_70_to_74', 'pop_75_to_79', 'pop_80_to_84', 'pop_85_to_89',
       'pop_90_to_94', 'pop_95_to_99', 'pop_100_plus', 'household_income',
       'income_under_5000', 'income_between_5000_9999',
       'inco

In [434]:
census2016.drop(columns=['total_highest_education'], inplace=True)

In [438]:
census2016.to_csv('data/census2016.csv')

In [437]:
census2016.shape

(141, 63)

# Check if neighbourhoods match

In [402]:
neighbourhoods = pd.read_csv('data/neighbourhoods.csv', index_col=0)

In [403]:
neighbourhoods.head()

,past_neighbourhood_158,neighbourhood
0,Agincourt North,Agincourt North
1,Agincourt South-Malvern West,Agincourt South-Malvern West
2,Alderwood,Alderwood
3,Annex,Annex
4,Avondale,Willowdale East


In [408]:
neighbourhoods['neighbourhood'] = neighbourhoods['neighbourhood'].str.lower()
neighbourhoods['neighbourhood'] = neighbourhoods['neighbourhood'].map(lambda x: x.translate(str.maketrans('', '', string.punctuation)))

In [409]:
neighbourhoods['neighbourhood'].equals(census2016.index.to_frame())

False

In [414]:
for row in census2016.index:
    print(row)


city of toronto
agincourt north
agincourt southmalvern west
alderwood
annex
banburydon mills
bathurst manor
bay street corridor
bayview village
bayview woodssteeles
bedford parknortown
beechboroughgreenbrook
bendale
birchcliffecliffside
black creek
blakejones
briar hillbelgravia
bridle pathsunnybrookyork mills
broadview north
brookhavenamesbury
cabbagetownsouth st james town
caledoniafairbank
casa loma
centennial scarborough
churchyonge corridor
clairleabirchmount
clanton park
cliffcrest
corso italiadavenport
danforth
danforth east york
don valley village
dorset park
dovercourtwallace emersonjunction
downsviewrodingcfb
dufferin grove
east enddanforth
edenbridgehumber valley
eglinton east
elmsold rexdale
englemountlawrence
eringatecentennialwest deane
etobicoke west mall
flemingdon park
forest hill north
forest hill south
glenfieldjane heights
greenwoodcoxwell
guildwood
henry farm
high park north
high parkswansea
highland creek
hillcrest village
humber heightswestmount
humber summit
hum

In [418]:
neighbourhoods[neighbourhoods['neighbourhood'] == 'cabbagetownsouth stjames town'] = 'cabbagetownsouth st james town'

In [419]:
neighbourhoods.to_csv('data/neighbourhoods.csv')

# Crime cleaning

In [421]:
crimes = pd.read_csv('data/crime/major_crimes.csv', index_col=0)

In [422]:
crimes['NEIGHBOURHOOD_140'] = crimes['NEIGHBOURHOOD_140'].str.lower()
crimes['NEIGHBOURHOOD_140'] = crimes['NEIGHBOURHOOD_140'].map(lambda x: x.translate(str.maketrans('', '', string.punctuation)))

In [426]:
crimes.to_csv('data/crime/major_crimes.csv')

# Make total population

In [491]:
census2011 = pd.read_csv('data/census2011.csv', index_col=0)
census2016 = pd.read_csv('data/census2016.csv', index_col=0)
census2021 = pd.read_csv('data/census2021.csv', index_col=0)

In [501]:
census2011['total_population'] = census2011[['pop_0_to_4', 'pop_5_to_9', 'pop_10_to_14', 'pop_15_to_19', 'pop_20_to_24', 
                             'pop_25_to_29', 'pop_30_to_34', 'pop_35_to_39', 'pop_40_to_44', 'pop_45_to_49', 
                             'pop_50_to_54', 'pop_55_to_59', 'pop_60_to_64', 'pop_65_to_69', 'pop_70_to_74', 
                             'pop_75_to_79', 'pop_80_to_84', 'pop_85_to_89', 'pop_90_to_94', 'pop_95_to_99', 
                             'pop_100_plus']].sum(axis=1)

In [511]:
census2016['total_population'] = census2016[['pop_0_to_4', 'pop_5_to_9', 'pop_10_to_14', 'pop_15_to_19', 'pop_20_to_24', 
                             'pop_25_to_29', 'pop_30_to_34', 'pop_35_to_39', 'pop_40_to_44', 'pop_45_to_49', 
                             'pop_50_to_54', 'pop_55_to_59', 'pop_60_to_64', 'pop_65_to_69', 'pop_70_to_74', 
                             'pop_75_to_79', 'pop_80_to_84', 'pop_85_to_89', 'pop_90_to_94', 'pop_95_to_99', 
                             'pop_100_plus']].sum(axis=1)

In [503]:
census2021['total_population'] = census2021[['pop_0_to_4', 'pop_5_to_9', 'pop_10_to_14', 'pop_15_to_19', 'pop_20_to_24', 
                             'pop_25_to_29', 'pop_30_to_34', 'pop_35_to_39', 'pop_40_to_44', 'pop_45_to_49', 
                             'pop_50_to_54', 'pop_55_to_59', 'pop_60_to_64', 'pop_65_to_69', 'pop_70_to_74', 
                             'pop_75_to_79', 'pop_80_to_84', 'pop_85_to_89', 'pop_90_to_94', 'pop_95_to_99', 
                             'pop_100_plus']].sum(axis=1)

In [513]:
census2011.to_csv('data/census2011.csv')
census2016.to_csv('data/census2016.csv')
census2021.to_csv('data/census2021.csv')

# Crime concatenation

In [617]:
census2011 = pd.read_csv('data/census2011.csv')
census2016 = pd.read_csv('data/census2016.csv')
census2021 = pd.read_csv('data/census2021.csv')

In [532]:
df_2013 = pd.read_csv('data/crime/by_neighbourhood/2013crimes_per_neighbourhood.csv')
df_2014 = pd.read_csv('data/crime/by_neighbourhood/2014crimes_per_neighbourhood.csv')
df_2015 = pd.read_csv('data/crime/by_neighbourhood/2015crimes_per_neighbourhood.csv')

# Concatenate the dataframes
crimes_census2011 = pd.concat([df_2013, df_2014, df_2015])
crime_totals_2011 = crimes_census2011.groupby('NEIGHBOURHOOD_140')['num_crimes'].sum().reset_index()

In [533]:
df_2016 = pd.read_csv('data/crime/by_neighbourhood/2016crimes_per_neighbourhood.csv')
df_2017 = pd.read_csv('data/crime/by_neighbourhood/2017crimes_per_neighbourhood.csv')
df_2018 = pd.read_csv('data/crime/by_neighbourhood/2018crimes_per_neighbourhood.csv')
df_2019 = pd.read_csv('data/crime/by_neighbourhood/2019crimes_per_neighbourhood.csv')
df_2020 = pd.read_csv('data/crime/by_neighbourhood/2020crimes_per_neighbourhood.csv')

# Concatenate the dataframes
crimes_census2016 = pd.concat([df_2016, df_2017, df_2018, df_2019, df_2020])
crime_totals_2016 = crimes_census2016.groupby('NEIGHBOURHOOD_140')['num_crimes'].sum().reset_index()

In [534]:
df_2021 = pd.read_csv('data/crime/by_neighbourhood/2021crimes_per_neighbourhood.csv')
df_2022 = pd.read_csv('data/crime/by_neighbourhood/2022crimes_per_neighbourhood.csv')
df_2023 = pd.read_csv('data/crime/by_neighbourhood/2023crimes_per_neighbourhood.csv')

# Concatenate the dataframes
crimes_census2021 = pd.concat([df_2021, df_2022, df_2023])
crime_totals_2021 = crimes_census2021.groupby('NEIGHBOURHOOD_140')['num_crimes'].sum().reset_index()

In [538]:
crime_totals_2011.rename(columns={'NEIGHBOURHOOD_140': 'neighbourhood'}, inplace=True)
crime_totals_2016.rename(columns={'NEIGHBOURHOOD_140': 'neighbourhood'}, inplace=True)
crime_totals_2021.rename(columns={'NEIGHBOURHOOD_140': 'neighbourhood'}, inplace=True)

In [573]:
crime_totals_2011 = pd.read_csv('data/crime/by_neighbourhood/crime_totals_2011.csv')
crime_totals_2016 = pd.read_csv('data/crime/by_neighbourhood/crime_totals_2016.csv')
crime_totals_2021 = pd.read_csv('data/crime/by_neighbourhood/crime_totals_2021.csv')

In [619]:
census2011 = census2011.merge(crime_totals_2011, how='left', on='neighbourhood')

In [632]:
census2016 = census2016.merge(crime_totals_2016, on='neighbourhood', how='left')

In [622]:
census2021 = census2021.merge(crime_totals_2021, on='neighbourhood', how='left')

In [634]:
census2016.drop(columns=['Unnamed: 0'], inplace=True)

In [635]:
census2011.to_csv('data/census2011.csv')
census2016.to_csv('data/census2016.csv')
census2021.to_csv('data/census2021.csv')